# Week2_4 Assignment

## [BASIC](#Basic) 
- 커스텀 모듈(`helper.py`)에서 **클래스와 함수를 임포트**할 수 있다.
- **autograd**의 개념 복습


## [CHALLENGE](#Challenge)
- train() 함수에 **epoch, scheduler, grad_clipping**을 추가할 수 있다.
- **validate() 함수를 구현**할 수 있다.


## [ADVANCED](#Advanced)
- train() 함수를 사용해 데이터를 **4 epoch 학습**할 수 있다. 
- **predict 함수를 구현**할 수 있다. 
- **evaluation metric 구현**할 수 있다. 
    - accuracy



### Reference
- [Pytorch Autograd Explain official document](https://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html)

In [ ]:
import os
import sys
import pandas as pd
import numpy as np 
import torch
import random

In [ ]:
# set seed
seed = 7777
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [ ]:
!pip install transformers

     |████████████████████████████████| 3.5 MB 7.8 MB/s 
     |████████████████████████████████| 895 kB 48.6 MB/s 
     |████████████████████████████████| 6.5 MB 31.0 MB/s 
     |████████████████████████████████| 67 kB 4.6 MB/s 
     |████████████████████████████████| 596 kB 33.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import files #뭐가 문제인지 구글드라이브를 통해 업로드한게 안먹혀서 직접 업로드했다
src = list(files.upload().values())[0] 
open('helper.py','wb').write(src) 


Saving helper.py to helper.py


3532

In [ ]:
# 어제 자신이 구현한 helper.py 모듈 경로를 입력
sys.path.append('/content/drive/MyDrive/AI/프리온보딩')

In [ ]:
# helper 모듈을 import하면 이전에 구현했던 다양한 함수 및 클래스를 사용할 수 있음 
# 함수: set_device()
# 함수: custom_collate_fn() 
# 클래스: CustomDataset
# 클래스: CustomClassifier
# 가 import 됨
from helper import *
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, random_split

Downloading:   0%|          | 0.00/289 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/425 [00:00<?, ?B/s]

In [ ]:
# device
device =set_device()
print(f"device: {device}")

# available GPUs : 1
GPU name : Tesla P100-PCIE-16GB
device: cuda


In [ ]:
device_cpu=torch.device('cpu') #cuda 메모리가 자꾸 아웃되서 학습이 끝날때마다 cpu로 옮겨주려고 만듬

## Basic

### 모듈에서 클래스와 함수를 임포트해 다음을 구현
- train_dataset, train_dataloader
- valid_dataset, valid_dataloader
- test_dataset, test_dataloader

In [ ]:
# train dataframe 다운로드
!wget https://raw.githubusercontent.com/ChristinaROK/PreOnboarding_AI_assets/e56006adfac42f8a2975db0ebbe60eacbe1c6b11/data/sample_df.csv

--2022-03-03 09:23:36--  https://raw.githubusercontent.com/ChristinaROK/PreOnboarding_AI_assets/e56006adfac42f8a2975db0ebbe60eacbe1c6b11/data/sample_df.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 971625 (949K) [text/plain]
Saving to: ‘sample_df.csv’

sample_df.csv       100%[===================>] 948.85K  --.-KB/s    in 0.03s   

2022-03-03 09:23:37 (27.6 MB/s) - ‘sample_df.csv’ saved [971625/971625]



In [ ]:
# test dataframe 다운로드
!wget https://raw.githubusercontent.com/ChristinaROK/PreOnboarding_AI_assets/main/data/sample_df_test.csv

--2022-03-03 09:23:37--  https://raw.githubusercontent.com/ChristinaROK/PreOnboarding_AI_assets/main/data/sample_df_test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 101383 (99K) [text/plain]
Saving to: ‘sample_df_test.csv’

sample_df_test.csv  100%[===================>]  99.01K  --.-KB/s    in 0.008s  

2022-03-03 09:23:37 (11.6 MB/s) - ‘sample_df_test.csv’ saved [101383/101383]



In [ ]:
# 학습 & 평가 데이터셋 로드
# 학습 및 평가 샘플 데이터 개수는 각각 10,000개, 1,000개

df_train = pd.read_csv('sample_df.csv')
df_test = pd.read_csv('sample_df_test.csv')

print(f"train shape : {df_train.shape}")
print(f"test shape : {df_test.shape}")

train shape : (10000, 3)
test shape : (1000, 3)


In [ ]:
df_train.columns #컬럼명 확인

Index(['id', 'document', 'label'], dtype='object')

In [ ]:
# Dataset 구현
# helper.py에 있는 CustomDataset 활용하여 train datset, test dataset 만들기
"""
    - input_data: list of string
    - target_data: list of int
    """
train_dataset = CustomDataset(df_train['document'],df_train['label']) #customdataset에 text list와 target list를 입력
test_dataset = CustomDataset(df_test['document'],df_test['label'])

print(f"Train Dataset len: {len(train_dataset)}")
print(f"Train Dataset 1st element: {train_dataset[0]}")

print(f"Test Dataset len: {len(test_dataset)}")
print(f"Test Dataset 1st element: {test_dataset[0]}")


Train Dataset len: 10000
Train Dataset 1st element: ('나 이거 더빙을 누가하는지 모르고 봤는데 왠지 더빙이 구리더라...더빙이 너무 별로였음.', 0)
Test Dataset len: 1000
Test Dataset 1st element: ('신용문객잔 보고 후속편인줄 알고 봤더만 완전 개판이네 18.. 이련결 그냥 절에나 쳐 들어 가라.. 회오리에서 싸우는 신 참 가관이더라 .. 서극도 완전 쓰레기 감독이 다 됐구나.. 액션도 쓰레기고 배우들 연기도 참 가관이더라 18', 0)


In [ ]:
# Train Dataset을 학습과 검증 셋으로 분리
# 학습 셋과 검증 셋의 비율은 9:1
# torch.utils.data에서 제공되는 데이터 세트를 임의로 분할할 수 있는 함수 찾아서 사용
n_train_sample = df_train.shape[0]

n_train = int(n_train_sample*0.9)
n_valid = n_train_sample - n_train 
train_dataset, valid_dataset = torch.utils.data.random_split(train_dataset,[n_train,n_valid]) #train valid split

print(f"Train dataset len: {len(train_dataset)}")
print(f"Valid dataset len: {len(valid_dataset)}")

Train dataset len: 9000
Valid dataset len: 1000


In [ ]:
# DataLoader 구현
# train과 validation의 batch size는 각각 32, 64로 설정
# test의 batch size는 validation과 동일
# train에 사용할 DataLoader에서는 sampler로 RandomSampler 사용
# validation과 test에 사용할 DataLoader에서는 sampler로 SequentialSampler 사용
# 모든 DataLoader의 collate_fn은 helper.py에 있는 custom_collate_fn 사용

train_batch_size = 32
valid_batch_size = 64
#dataset 별 dataloader 생성
train_dataloader = DataLoader(train_dataset,batch_size=train_batch_size,sampler=RandomSampler(train_dataset),collate_fn=custom_collate_fn) 

valid_dataloader = DataLoader(valid_dataset,batch_size=valid_batch_size,sampler=SequentialSampler(valid_dataset),collate_fn=custom_collate_fn)

test_dataloader = DataLoader(test_dataset,batch_size=valid_batch_size,sampler=SequentialSampler(test_dataset),collate_fn=custom_collate_fn)

print(f"Train dataloader # steps: {len(train_dataloader)}")
print(f"Valid dataloader # steps: {len(valid_dataloader)}")
print(f"Test dataloader # steps: {len(test_dataloader)}")

Train dataloader # steps: 282
Valid dataloader # steps: 16
Test dataloader # steps: 16


### `auto_grad` 개념 복습
- torch의 `auto_grad` 기능
    - pytorch는 `requires_grad` 파리미터의 값이 True인 텐서에 한해서 미분값을 자동으로 계산한다.
    - 미분값은 `loss.backward()` 가 호출될 때 자동으로 계산된다.

In [ ]:
# helper.py에 있는 CustomClassifier 모델을 로드해 model_unfreeze 변수에 instance를 생성
hidden_size=768
n_label=2


model_freeze = CustomClassifier(hidden_size=hidden_size,n_label=n_label) #unfreeze model 생성

Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# model_freeze 모델의 모든 파라미터를 출력해보고 아래 질문에 답해 보자

In [ ]:
for name, param in model_freeze.named_parameters(): #model_freeze에있는 parameter의 이름과 parameter를 꺼냄
    print(f'name:{name}')  #파라미터 이름
    print(type(param)) #파라미터가 맞는지
    print(f'param.shape:{param.shape}') #shape
    print(f'param.requries_grad:{param.requires_grad}') #requires_gard 설정
    print(f'param.grad:{param.grad}') #갖고있는 grad
    print('=====')

name:bert.embeddings.word_embeddings.weight
<class 'torch.nn.parameter.Parameter'>
param.shape:torch.Size([32000, 768])
param.requries_grad:True
param.grad:None
=====
name:bert.embeddings.position_embeddings.weight
<class 'torch.nn.parameter.Parameter'>
param.shape:torch.Size([512, 768])
param.requries_grad:True
param.grad:None
=====
name:bert.embeddings.token_type_embeddings.weight
<class 'torch.nn.parameter.Parameter'>
param.shape:torch.Size([2, 768])
param.requries_grad:True
param.grad:None
=====
name:bert.embeddings.LayerNorm.weight
<class 'torch.nn.parameter.Parameter'>
param.shape:torch.Size([768])
param.requries_grad:True
param.grad:None
=====
name:bert.embeddings.LayerNorm.bias
<class 'torch.nn.parameter.Parameter'>
param.shape:torch.Size([768])
param.requries_grad:True
param.grad:None
=====
name:bert.encoder.layer.0.attention.self.query.weight
<class 'torch.nn.parameter.Parameter'>
param.shape:torch.Size([768, 768])
param.requries_grad:True
param.grad:None
=====
name:bert.enco

### `auto_grad` 개념 및 모델 구조 복습을 위해 다음 항목에 답해 보자
name:bert.encoder.layer.0.attention.self.query.weight  
<class 'torch.nn.parameter.Parameter'>  
param.shape:torch.Size([768, 768])  
param.requries_grad:True  
- `bert.encoder.layer.0.attention.self.query.weight` 텐서의 gradient는 True인 상태인가?
> 
> ## 정답 : **True**

name:classifier.0.weight  
<class 'torch.nn.parameter.Parameter'>  
param.shape:torch.Size([32, 768])  
param.requries_grad:True  
param.grad:None
- `classifier.0.weight` 텐서의 shape은? 
> ## 정답 : 32,768
- `classifier.0.weight` 텐서는 freeze 상태인가 ? 
> ## 정답 : unfreeze 상태이다
- `classifier.0.weight` 텐서의 gradient 값은 무엇인가? 
> ## 정답 : None

### 위 모델 (`model_freeze`)의 모든 파라미터의 gradient를 freeze 해보자

In [ ]:
# 모든 파라미터의 gradient를 freeze 해보고 제대로 변경되었는지 확인하기 위해 모델의 모든 파라미터를 출력해보자.

for name, param in model_freeze.named_parameters(): #model_freeze에있는 parameter의 이름과 parameter를 꺼냄
    print(f'name:{name}')  #파라미터 이름      
    param.requires_grad=False #freeze 로 변경
    print(f'param.requries_grad:{param.requires_grad}') #requires_gard 설정    
    print('=====')
    


name:bert.embeddings.word_embeddings.weight
param.requries_grad:False
=====
name:bert.embeddings.position_embeddings.weight
param.requries_grad:False
=====
name:bert.embeddings.token_type_embeddings.weight
param.requries_grad:False
=====
name:bert.embeddings.LayerNorm.weight
param.requries_grad:False
=====
name:bert.embeddings.LayerNorm.bias
param.requries_grad:False
=====
name:bert.encoder.layer.0.attention.self.query.weight
param.requries_grad:False
=====
name:bert.encoder.layer.0.attention.self.query.bias
param.requries_grad:False
=====
name:bert.encoder.layer.0.attention.self.key.weight
param.requries_grad:False
=====
name:bert.encoder.layer.0.attention.self.key.bias
param.requries_grad:False
=====
name:bert.encoder.layer.0.attention.self.value.weight
param.requries_grad:False
=====
name:bert.encoder.layer.0.attention.self.value.bias
param.requries_grad:False
=====
name:bert.encoder.layer.0.attention.output.dense.weight
param.requries_grad:False
=====
name:bert.encoder.layer.0.atte

## Challenge

### `scheduler` 를 생성 
- 스케쥴러를 알기 전에 먼저 `epoch`의 개념을 알아야 한다. Epoch는 dataset를 **몇 번 반복**해 학습할 것인지를 의미한다. 만약 dataset의 개수가 2,000개이고 epoch을 2번 학습하게 되면 총 4,000개의 데이터를 학습하게 된다.   
- 스케쥴러는 epoch에 따라 learning rate의 값을 조정하는 것을 의미한다. 
- 예를 들어 [여기](https://huggingface.co/docs/transformers/main_classes/optimizer_schedules#transformers.get_linear_schedule_with_warmup)의 그림에서 볼 수 있듯이 `get_linear_schedule_with_warmup`는 특정 step까지는 learning rate를 천천히 상승시키다가 고점에 도달하면 다시 하락시킨다. 

### `model`, `optimizer`, `scheduler`를 초기화(=인스턴스 생성)하는 함수를 구현하라

In [ ]:
from torch.nn import CrossEntropyLoss
from torch.optim import AdamW
from torch.nn.utils import clip_grad_norm_
from transformers import get_linear_schedule_with_warmup, get_constant_schedule

In [ ]:
# model:CustomClassifier 사용, hidden size는 768, label 개수는 2
# optimizer: AdamW 사용, learning rate는 2e-5
# scheduler: transformers.get_linear_schedule_with_warmup 함수 사용, 단, num_warmup_steps 매개 변수는 사용하지 않음

def initializer(train_dataloader, epochs=2):
    """
    모델, 옵티마이저, 스케쥴러를 초기화한 후 반환
    """
    
    model = CustomClassifier(768,2)

    optimizer = AdamW(model.parameters(),
                      lr=2e-5)
                                        
    
    total_steps = len(train_dataloader) * epochs
    print(f"Total train steps with {epochs} epochs: {total_steps}")

    scheduler = get_linear_schedule_with_warmup(optimizer=optimizer,
                                                num_warmup_steps = 0, #아예 안써버리면 오에러남
                                                num_training_steps=total_steps)#epoch말고 토탈 스텝을 넣어줘야한다..

    return model, optimizer, scheduler

### model, optimizer, scheduler의 파라미터 저장하는 함수를 구현하라

In [ ]:
model_freeze.state_dict

<bound method Module.state_dict of CustomClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768

In [ ]:
# 모델 저장 함수 구현
#https://discuss.pytorch.org/t/how-to-save-and-load-lr-scheduler-stats-in-pytorch/20208
def save_checkpoint(path, model, optimizer, scheduler, epoch, loss):
    file_name = f'{path}/model.ckpt.{epoch}'
    
    # torch.save 함수 참고
    torch.save(
        {
            'epoch': epoch,
            'model_state_dict': model.state_dict(), 
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(), #()을 꼭 써주자..
            'loss' : loss
        }, 
        file_name
    )
    
    print(f"Saving epoch {epoch} checkpoint at {file_name}")

### `validate()` 함수 구현 
- `validate()` 함수 내 model의 상태는 **evaluate**이어야 한다. evaluate 상태의 model은 dropout을 진행하지 않는다. 
- **forward**를 진행할 때 `with torch.no_grad(): ...` 설정해 미분 계산을 방지한다.


In [ ]:
import torch.nn.functional as F

In [ ]:
# input: model, valid_dataloader
# output: loss, 정확도

def validate(model, valid_dataloader):
    global loss_fct
   
    # 모델을 evaluate 모드로 설정 & device 할당
    model.eval()
    
    total_loss, total_acc= 0,0
        
    for step, batch in enumerate(valid_dataloader):
        
        # tensor 연산 전, 각 tensor에 device 할당
        batch = tuple(item.to(device) for item in batch)
            
        batch_input, batch_label = batch
            
        # gradient 계산하지 않고 forward 진행
        with torch.no_grad():  #검증이라 막아줘야한다
            logits = model.forward(**batch_input)
            
        # loss
        loss = loss_fct(logits, batch_label)
        total_loss += loss.item()
        
        # accuracy
        probs = F.softmax(logits, dim=1)
        preds = torch.argmax(probs, dim=1).flatten()
        acc = (preds == batch_label).cpu().numpy().mean()
        total_acc+=acc
        batch_input.to(device_cpu) #메모리 아웃을 방지하기위해 cpu로 옮김
        batch_label.to(device_cpu)

    
    total_loss = total_loss/(step+1)
    total_acc = total_acc/(step+1)*100

    return total_loss, total_acc


### `train()` 함수에 `epoch`와 `clip_grad_norm` 추가
- data_loader를 `epoch`만큼 반복하면서 학습하도록 `train()` 함수를 수정하라
- `gradient cliping`은 미분 값 너무 큰 경우 gradient exploding되는 현상을 막기 위해 미분값이 `threshold`를 넘을 경우 특정 비율을 미분 값에 곱해 크기를 줄여준다.
- Reference
  - [clip_grad_norm_ official document](https://pytorch.org/docs/stable/generated/torch.nn.utils.clip_grad_norm_.html)
  - [그래디언트 클립핑 설명 한국어 블로그](https://kh-kim.gitbook.io/natural-language-processing-with-pytorch/00-cover-6/05-gradient-clipping)

In [ ]:
from torch.nn.utils.clip_grad import clip_grad_norm
# 위에서 구현한 모델 저장 함수(save_checkpoint)와 validate 함수도 추가해보자

loss_fct = CrossEntropyLoss()

def train(model, train_dataloader, valid_dataloader=None, epochs=2):
        global scheduler, loss_fct
        
        # train_dataloaer 학습을 epochs만큼 반복
        for epoch in range(epochs):
            print(f"*****Epoch {epoch} Train Start*****")
            
            # 배치 단위 평균 loss와 총 평균 loss 계산하기위해 변수 생성
            total_loss, batch_loss, batch_count = 0,0,0
        
            # model을 train 모드로 설정 & device 할당
            model.train()
            model.to(device)
            
            # data iterator를 돌면서 하나씩 학습
            for step, batch in enumerate(train_dataloader):
                batch_count+=1
                
                # tensor 연산 전, 각 tensor에 device 할당
                batch = tuple(item.to(device) for item in batch)
            
                batch_input, batch_label = batch
            
                # batch마다 모델이 갖고 있는 기존 gradient를 초기화
                model.zero_grad()
            
                # forward
                logits = model(**batch_input)
            
                # loss
                loss = loss_fct(logits, batch_label)
                batch_loss += loss.item()
                total_loss += loss.item()
            
                # backward -> 파라미터의 미분(gradient)를 자동으로 계산
                loss.backward()
                
                # gradient clipping 적용 (max_norm = 1)
                clip_grad_norm(model.parameters(), 
                               max_norm=1)
                
                # optimizer & scheduler 업데이트
                optimizer.step() #optimizer와 scheduler 둘다 step을 줘야한다
                scheduler.step()
                
                # 배치 10개씩 처리할 때마다 평균 loss와 lr를 출력
                if (step % 10 == 0 and step != 0):
                    learning_rate = optimizer.param_groups[0]['lr']
                    print(f"Epoch: {epoch}, Step : {step}, LR : {learning_rate}, Avg Loss : {batch_loss / batch_count:.4f}")

                    # reset 
                    batch_loss, batch_count = 0,0
                batch_input.to(device_cpu)#메모리 아웃 방지 to cpu
                batch_label.to(device_cpu)    

            print(f"Epoch {epoch} Total Mean Loss : {total_loss/(step+1):.4f}")
            print(f"*****Epoch {epoch} Train Finish*****\n")
            
            if valid_dataloader is not None:
                print(f"*****Epoch {epoch} Valid Start*****")
                valid_loss, valid_acc = validate(model,valid_dataloader)
                print(f"Epoch {epoch} Valid Loss : {valid_loss:.4f} Valid Acc : {valid_acc:.2f}")
                print(f"*****Epoch {epoch} Valid Finish*****\n")
            
            # checkpoint 저장
            save_checkpoint('/content/drive/MyDrive/AI/프리온보딩/2-4check',model,optimizer,scheduler,epoch,loss)
                
        print("Train Completed. End Program.")

## Advanced

### 학습 데이터를 epoch 4까지 학습
- 매 epoch마다 다음을 수행한다.
  - 학습이 끝난 후 validate() 함수 실행 
  - validate() 함수가 끝난 후 model save 함수 실행

In [ ]:
from transformers import BertTokenizer, BertModel

In [ ]:
tokenizer_bert = BertTokenizer.from_pretrained("klue/bert-base")

In [ ]:
# 4 epoch 학습
epochs=4
model, optimizer, scheduler = initializer(train_dataloader, epochs) #인스턴트 모델
train(model, train_dataloader, valid_dataloader, epochs)

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Total train steps with 4 epochs: 1128
*****Epoch 0 Train Start*****


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.


Epoch: 0, Step : 10, LR : 1.9804964539007094e-05, Avg Loss : 0.6573
Epoch: 0, Step : 20, LR : 1.962765957446809e-05, Avg Loss : 0.5602
Epoch: 0, Step : 30, LR : 1.945035460992908e-05, Avg Loss : 0.4979
Epoch: 0, Step : 40, LR : 1.927304964539007e-05, Avg Loss : 0.4326
Epoch: 0, Step : 50, LR : 1.9095744680851064e-05, Avg Loss : 0.3881
Epoch: 0, Step : 60, LR : 1.891843971631206e-05, Avg Loss : 0.4225
Epoch: 0, Step : 70, LR : 1.8741134751773053e-05, Avg Loss : 0.4150
Epoch: 0, Step : 80, LR : 1.8563829787234043e-05, Avg Loss : 0.3981
Epoch: 0, Step : 90, LR : 1.8386524822695038e-05, Avg Loss : 0.3459
Epoch: 0, Step : 100, LR : 1.8209219858156032e-05, Avg Loss : 0.3662
Epoch: 0, Step : 110, LR : 1.8031914893617023e-05, Avg Loss : 0.3939
Epoch: 0, Step : 120, LR : 1.7854609929078013e-05, Avg Loss : 0.4043
Epoch: 0, Step : 130, LR : 1.7677304964539008e-05, Avg Loss : 0.3979
Epoch: 0, Step : 140, LR : 1.7500000000000002e-05, Avg Loss : 0.3547
Epoch: 0, Step : 150, LR : 1.7322695035460996e-

### 가장 dev acc 성능이 높았던 epoch의 모델의 체크 포인트를 불러와 로드하자

In [ ]:
# torch.load 함수 사용

checkpoint = torch.load('/content/drive/MyDrive/AI/프리온보딩/2-4check/model.ckpt.3')

In [ ]:
# checkpoint의 key 종류를 확인
checkpoint.keys()

dict_keys(['epoch', 'model_state_dict', 'optimizer_state_dict', 'scheduler_state_dict', 'loss'])

In [ ]:
# 위에서 구현한 initializer 함수 사용하여 model, optimizer, scheduler 초기화

epochs=1
model, optimizer, scheduler = initializer(train_dataloader,epochs)

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Total train steps with 1 epochs: 282


In [ ]:
model.load_state_dict(checkpoint["model_state_dict"]) #체크포인트 로드

<All keys matched successfully>

### 모델 예측 함수 구현
- test_dataloader를 입력받아 모델이 예측한 확률값 (probs)과 실제 정답 (label) 을 출력하는 `predict()` 함수를 구현하자.
- 함수 정의
  - 입력 매개변수
    - `model` : `CustomClassifier` 모델. logits를 반환함 
    - `test_dataloader` : test 데이터셋의 텍스트와 레이블을 배치로 갖는 dataloader
  - 조건
    - `test_dataloader`는 이터레이터기 때문에 이터레이터를 순회하면서 `all_logits` 리스트에 배치 단위의 logits를 저장하고 `all_labels` 리스트에 배치 단위의 레이블 (0 또는 1 값)을 저장하라
  - 반환값
    - `probs`
      - logits에 softmax 함수를 취한 확률값. (test data 개수, label 개수) shape을 가짐. np.array 타입으로 데이터 타입을 변환할 것.
    - `labels`
      - 0 또는 1 값을 갖는 np.array. (test data 개수,) shape을 가짐.

In [ ]:
def predict(model, test_dataloader):
    """
    test_dataloader의 label별 확률값과 실제 label 값을 반환
    """

    # model을 eval 모드로 설정 & device 할당
    model.eval()
    model.to(device)

    all_logits = []
    all_labels = []

    for step, batch in enumerate(test_dataloader):
        
        batch_input, batch_label = batch
        
        # batch_input을 device 할당
        batch_input.to(device)

        # model에 batch_input을 넣어 logit 반환 & all_logits, all_labels 리스트에 값 추가 
        with torch.no_grad(): #검증과 마찬가지로 gredient 계산을 막음
            logits = model.forward(**batch_input)        

        all_logits.append(logits)
        all_labels.append(batch_label)

        logits.to(device_cpu)#메모리 아웃방지
   
    

    # logits을 확률값으로 변환 & Tensor 타입을 numpy.array 타입으로 변환      
    probs=F.softmax(torch.cat(all_logits,dim=0),dim=1).to(device_cpu).numpy() #모아진 tensor들을 concat => 확률화 => to cpu =>numpy
    
    all_labels =torch.cat(all_labels,dim=0).to(device_cpu).numpy()#  Tensor 타입을 numpy.array 타입으로 변환

    return probs, all_labels



- 모델이 예측한 확률값과 실제 label을 입력 받아 정확도를 출력하는 **accuracy()** 함수를 구현하자. 
- 함수 정의 
  - 입력 매개변수 
    - `probs` : `predict()` 함수의 반환값. 2차원의 np.array
    - `labels` : `predict()` 함수의 반환값. 1차원의 np.array
  - 조건
    - `probs`의 확률값이 0.5 이상이면 1, 이하이면 0이 되도록 만든다. 모델이 예측한 레이블을 실제값(`labels`)과 비교해 예측값과 실제값이 같으면 1, 다르면 0 점수를 준다. 모든 데이터에 대해 점수의 평균값이 accuracy 값이다. 
  - 반환값 
    - `acc` : 정확도 (Float type)

In [ ]:
# accuracy 함수 구현
def accuracy(probs, labels):   
    y_pred = np.where(probs[:,1] < 0.5, 0, 1) # probs(확률값)을 label로 변경(0.5 이상이면 1, 0.5 미만이면 0)     
    right,all=0,0    
    for pred,label in zip(y_pred,labels):#정확도 계산
        right += pred == label 
        all+=1
    acc=right/all
    return acc 


In [ ]:
probs, labels = predict(model, test_dataloader)

In [ ]:
threshold =0.5
accuracy(probs,labels)

0.863

### `sklearn.metrics`의 `accuracy_score`, `roc_auc_score` 함수를 이용해 정확도와 auc를 계산하라

In [ ]:
from sklearn.metrics import roc_auc_score, accuracy_score

In [ ]:
# 정확도 출력
accuracy_score(labels,np.where(probs[:,1] < 0.5, 0, 1))
#만든 accuracy함수 0.863과 동일

0.863

In [ ]:
# auc 출력
roc_auc_score(labels,probs[:,1])

0.932226